<a href="https://colab.research.google.com/github/musman2012/ml-data_science/blob/master/SOH_Prediction/Bayes_Binning_BatteryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Phase-II
This phase would have the following keys steps to perform:
* Apply labelling/binning on the data (yet to decided the technique)
* Make three clusters/subsets of the data on the basis of cycle variable
* Apply Association Rule Mining on the subsets
* Apply prediction technique to classify the target variable soh
* Evaluate Models Results

More explanation for the process and what I want to acheive 

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import platform
import dill
dill.load_session('phase-II-ml-copy.db')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
'''
authored by Muhammad Usman (musman14@student.bradford.ac.uk)

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

FILE_PATH = "batteries_processed.csv"
bat_data = pd.read_csv(FILE_PATH, error_bad_lines = False)    # reading the data from csv file

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Removing the same features which were removed in Phase-I as well

In [3]:
filtered_data = bat_data.copy()
columns_to_drop = ['id', 'vin', 'vin_prefix', 'record_id','recorded_time', 'actual_time', 'arrival_time', 'powermode', 'mheader_vin', 'mheader_time', 'mheader_type', 'message','cat4', 'cat6', 'cat7', 'inlet_coolant_temp','cooling_energy_used', 'max_soh_cell', 'fuse_temp', 'power_soh', 'min_soh', 'max_soh', 'min_soh_cell_id','imbalance_percent', 'parkingdata_hvbattavgsocoatevent', 'parkingdata_hvbattavtempatevent']
filtered_data = filtered_data.drop(columns=columns_to_drop)
filtered_data.head()

,odometer,ambient_temp,vehicle_speed,balancing_trigger_cell,balancing_status,min_voltage,min_temp,traction_current,max_voltage,imbalance,traction_voltage,min_temp_cell,min_voltage_cell,max_voltage_cell,max_temp,max_temp_cell,emodule_temp,thermal_manager_mode,outlet_coolant_temp,soh,fast_charge_count,cycle,num_cycle,fast_charge,trip_distance
0,10619,6.5,0.0,22,noBalancing,4.107,12,0.675,4.153,0.046,445,15,22,96,14,1,135,activeHeating,195,98.0,0,storage,1,False,0
1,9531,4.0,0.0,22,noBalancing,3.983,6,7.375,4.008,0.025,432,16,36,21,7,1,90,idle,55,98.0,0,storage,1,False,-1088
2,9531,4.0,0.0,22,noBalancing,3.989,6,7.600,4.023,0.034,431,16,36,76,7,1,90,idle,55,98.0,0,storage,1,False,0
3,9531,4.0,0.0,22,noBalancing,3.984,6,6.850,4.020,0.036,431,16,36,79,7,1,90,idle,55,98.0,0,storage,1,False,0
4,9531,4.0,0.0,22,noBalancing,3.980,6,7.000,4.015,0.035,431,16,36,85,7,1,90,idle,55,98.0,0,storage,1,False,0


In [4]:
for c in filtered_data.columns:
  print("Column", c)
  print(filtered_data[c].value_counts(normalize='true'))

Column odometer
6923     3.802000e-04
3297     3.316000e-04
2739     3.222000e-04
3296     3.138000e-04
703      3.076000e-04
             ...     
17205    2.000000e-07
20739    2.000000e-07
29491    2.000000e-07
23197    2.000000e-07
21109    2.000000e-07
Name: odometer, Length: 23959, dtype: float64
Column ambient_temp
 9.25     3.626360e-02
 8.75     3.488080e-02
 8.25     3.181080e-02
 7.75     3.039420e-02
 9.75     2.789920e-02
              ...     
-20.50    2.000000e-07
-20.25    2.000000e-07
-21.50    2.000000e-07
-21.00    2.000000e-07
 43.50    2.000000e-07
Name: ambient_temp, Length: 266, dtype: float64
Column vehicle_speed
0.00      8.465960e-01
114.52    3.068000e-04
98.94     2.968000e-04
104.79    2.930000e-04
0.73      2.876000e-04
              ...     
168.86    2.000000e-07
162.22    2.000000e-07
154.74    2.000000e-07
163.91    2.000000e-07
165.31    2.000000e-07
Name: vehicle_speed, Length: 3037, dtype: float64
Column balancing_trigger_cell
49     0.024991
76   

### Binning the data
We have to choices for Binning the data here:
* Fixed-Width Binning: Specific fixed widths for each of the bins which are usually pre-defined
* Adaptive Binning: Such binning in which we use the data distribution itself to decide bin ranges

&rightarrow; Quantile Binning is one of the type of Adaptive Binning which helps in partitioning the continuous valued distribution of a specific numeric field into discrete contiguous bins or intervals. We will use this binning to convert out continuous variables into categorical variables.

In [0]:
def bayesian_blocks(t):
    """Bayesian Blocks Implementation

    By Jake Vanderplas.  License: BSD
    Based on algorithm outlined in http://adsabs.harvard.edu/abs/2012arXiv1207.5578S

    Parameters
    ----------
    t : ndarray, length N
        data to be histogrammed

    Returns
    -------
    bins : ndarray
        array containing the (N+1) bin edges

    Notes
    -----
    This is an incomplete implementation: it may fail for some
    datasets.  Alternate fitness functions and prior forms can
    be found in the paper listed above.
    """
    # copy and sort the array
    t = np.sort(t)
    t = np.unique(t)
    N = t.size

    # create length-(N + 1) array of cell edges
    edges = np.concatenate([t[:1],
                            0.5 * (t[1:] + t[:-1]),
                            t[-1:]])
    block_length = t[-1] - edges

    print(block_length[:1] , block_length[1])

    # arrays needed for the iteration
    nn_vec = np.ones(N)
    best = np.zeros(N, dtype=float)
    last = np.zeros(N, dtype=int)

    #-----------------------------------------------------------------
    # Start with first data cell; add one cell at each iteration
    #-----------------------------------------------------------------
    for K in range(N):
        # Compute the width and count of the final bin for all possible
        # locations of the K^th changepoint
        width = block_length[:K + 1] - block_length[K + 1]
        count_vec = np.cumsum(nn_vec[:K + 1][::-1])[::-1]
        #print("Width is {} and count_vec is {}", width, count_vec)

        # evaluate fitness function for these possibilities
        fit_vec = count_vec * (np.log(count_vec) - np.log(width))
        fit_vec -= 4  # 4 comes from the prior on the number of changepoints
        fit_vec[1:] += best[:K]

        # find the max of the fitness: this is the K^th changepoint
        i_max = np.argmax(fit_vec)
        last[K] = i_max
        best[K] = fit_vec[i_max]

    #-----------------------------------------------------------------
    # Recover changepoints by iteratively peeling off the last block
    #-----------------------------------------------------------------
    change_points =  np.zeros(N, dtype=int)
    i_cp = N
    ind = N
    while True:
        i_cp -= 1
        change_points[i_cp] = ind
        if ind == 0:
            break
        ind = last[ind - 1]
    change_points = change_points[i_cp:]

    return edges[change_points]

In [18]:
int_cols = filtered_data.select_dtypes('int64') ## finding all the continuous variables
float_cols = filtered_data.select_dtypes('float')
feature_bins = dict()

for col in int_cols.columns:
  feature_bins[col] = bayesian_blocks(filtered_data[col])

for col in float_cols.columns:
  feature_bins[col] = bayesian_blocks(filtered_data[col])

[142256.] 142252.0
[108.] 107.5
[91.] 79.0
[450.] 293.5
[107.] 106.0
[107.] 106.5
[107.] 106.5
[95.] 82.0
[107.] 106.0
[1025.] 1022.5
[1030.] 1027.5
[200.] 199.5
[246.] 245.5
[105578.] 88018.0
[174.5] 130.5
[200.13] 199.765
[4.155] 2.8920000000000003
[419430.375] 314765.375
[4.183] 2.6529999999999996
[1.161] 1.1605
[100.] 89.65


In [19]:
for k in feature_bins:
  print("******", k, "******")
  print(feature_bins[k])

****** odometer ******
[     0.   17722.5  17790.5  19474.5  19500.5  19531.5  19579.5  20124.5
  20166.5  20254.5  20421.   21561.5  21581.5  21756.5  21774.5  21829.5
  21860.5  21897.5  21981.5  21998.5  22036.5  22068.5  22096.5  22434.5
  22519.5  22597.5  22657.5  22670.   22703.5  22743.5  22821.5  22920.5
  23026.5  23103.   23193.5  23200.5  23250.5  23259.5  23363.5  23483.5
  23574.5  23596.5  23691.5  23717.5  23821.   23898.5  23934.5  23952.5
  24031.5  24055.5  24189.5  24197.5  24367.5  24384.5  24478.5  24575.
  24692.5  24726.5  24931.5  24951.5  25125.5  25152.5  25230.5  25246.5
  25353.5  25422.5  25439.5  25459.5  25475.5  25541.5  25548.5  25646.5
  25656.   25811.5  25835.5  25856.5  25911.5  25959.5  26024.   26387.5
  26420.5  26659.5  26779.5  26794.5  26890.5  26900.5  27013.5  27042.5
  27146.   27196.5  27312.5  27331.5  27355.5  27365.5  27420.5  27479.5
  27538.5  27546.5  27697.5  27701.5  27778.5  27872.5  27960.5  28004.5
  28088.5  28095.5  28129.5  

In [0]:
categ_data = pd.DataFrame()

for col in filtered_data.columns:
  count = 1
  

In [0]:
import dill
dill.dump_session('phase-II-ml-copy.db')

There are two problems in this:


1.   Process results are not very good, which is mostly because of not comprehensive binning technique applied. That is, the same combination of inputs are resulting Low and the same combination is resulting Medium SOH.
2.   In real world, we wont be getting the values in categories but would be cont. variables. So this method has limitations in that regard as well.